In [1]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

In [2]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [41]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [25]:
pd.options.display.max_columns=100
df.loc[54012:54013]

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
54012,2021-22,1610612744,GSW,Golden State Warriors,0022100169,2021-11-10,GSW vs. MIN,W,240,40,85,0.471,11,40,0.275,32,40,0.80,20,33,53,23,14,6,16,21,123,13,124.7,121.8,106.0,110.0,18.8,11.8,0.575,1.44,16.2,0.489,0.600,0.551,16.227,15.8,0.535,0.599,1.0,0.204,101.2,100.5,83.75,101.0,0.607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54013,2021-22,1610612750,MIN,Minnesota Timberwolves,0022100169,2021-11-10,MIN @ GSW,L,240,39,96,0.406,17,44,0.386,15,20,0.75,18,23,41,30,8,4,17,28,110,-13,106.0,110.0,124.7,121.8,-18.8,-11.8,0.769,1.76,19.8,0.400,0.511,0.449,16.378,17.0,0.495,0.525,1.0,0.200,101.2,100.5,83.75,100.0,0.393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
home_team = 'ATL'
away_team = 'BOS'
home_spread = -4
away_spread = -home_spread
home_ml = 1.57
away_ml = 2.55

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

matchup

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.14it/s]
progress:: 418it [00:00, 5037.20it/s]


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,AST_team_diff,STL_team_diff,BLK_team_diff,TOV_team_diff,PF_team_diff,PTS_team_diff,PLUS_MINUS_team_diff,E_OFF_RATING_team_diff,E_DEF_RATING_team_diff,E_NET_RATING_team_diff,POSS_team_diff,PIE_team_diff,PTS_2PT_MR_team_diff,PTS_FB_team_diff,PTS_OFF_TOV_team_diff,PTS_PAINT_team_diff,AST_2PM_team_diff,AST_3PM_team_diff,UAST_2PM_team_diff,UAST_3PM_team_diff,ATS_DIFF_team_diff,RECORD_opp_diff,FG2M_opp_diff,FG2A_opp_diff,FG3M_opp_diff,FG3A_opp_diff,FTM_opp_diff,FTA_opp_diff,OREB_opp_diff,DREB_opp_diff,REB_opp_diff,AST_opp_diff,STL_opp_diff,BLK_opp_diff,TOV_opp_diff,PF_opp_diff,PTS_opp_diff,PLUS_MINUS_opp_diff,E_OFF_RATING_opp_diff,E_DEF_RATING_opp_diff,E_NET_RATING_opp_diff,POSS_opp_diff,PIE_opp_diff,PTS_2PT_MR_opp_diff,PTS_FB_opp_diff,PTS_OFF_TOV_opp_diff,PTS_PAINT_opp_diff,AST_2PM_opp_diff,AST_3PM_opp_diff,UAST_2PM_opp_diff,UAST_3PM_opp_diff,ATS_DIFF_opp_diff,TEAM_COVERED_opp_diff,WL_PCT_diff,ATS_PCT_diff,AVG_SPREAD_diff,AVG_ML_diff,AVG_ATS_DIFF_diff,REST_diff,OREB_PCT_team_diff,OREB_PCT_opp_diff,DREB_PCT_team_diff,DREB_PCT_opp_diff,REB_PCT_team_diff,REB_PCT_opp_diff,TS_PCT_team_diff,TS_PCT_opp_diff,EFG_PCT_team_diff,EFG_PCT_opp_diff,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,-0.184584,0.135586,3.647106,0.689955,-2.466272,-0.867124,-1.073048,0.399349,-0.873349,-0.474,0.012331,-1.730853,-2.324553,-2.132217,0.45474,1.473913,-6.174031,3.425257,9.088508,-5.664829,-2.576806,-0.052041,1.382571,0.48709,-0.593177,-0.729183,0.42536,-0.892352,-0.225621,1.472456,-4.224976,0.184584,1.117481,-1.539853,2.45367,2.091635,-1.948029,-2.504743,-1.963123,-1.52202,-3.485143,5.482076,-0.230538,-0.060837,-2.730315,0.623395,7.647944,6.174031,9.088508,3.425257,5.664829,-2.05783,0.052041,0.468083,3.499887,-1.838306,1.533655,2.465885,2.471566,-1.260818,-0.314953,4.224976,0.304486,-0.104396,-0.252747,0.68956,0.152348,-2.645604,0.0,0.014719,-0.031594,0.031594,-0.014719,0.017397,-0.017397,0.002142,0.041699,0.006338,0.051582,0.575346,6.004677,-1.743846,-2.112105,0.645249,1537.223355,1502.302094,3.71,-4.0,1.57
1,0.184584,-0.135586,-3.647106,-0.689955,2.466272,0.867124,1.073048,-0.399349,0.873349,0.474,-0.012331,1.730853,2.324553,2.132217,-0.45474,-1.473913,6.174031,-3.425257,-9.088508,5.664829,2.576806,0.052041,-1.382571,-0.48709,0.593177,0.729183,-0.42536,0.892352,0.225621,-1.472456,4.224976,-0.184584,-1.117481,1.539853,-2.45367,-2.091635,1.948029,2.504743,1.963123,1.52202,3.485143,-5.482076,0.230538,0.060837,2.730315,-0.623395,-7.647944,-6.174031,-9.088508,-3.425257,-5.664829,2.05783,-0.052041,-0.468083,-3.499887,1.838306,-1.533655,-2.465885,-2.471566,1.260818,0.314953,-4.224976,-0.304486,0.104396,0.252747,-0.68956,-0.152348,2.645604,0.0,-0.014719,0.031594,-0.031594,0.014719,-0.017397,0.017397,-0.002142,-0.041699,-0.006338,-0.051582,-0.575346,-6.004677,1.743846,2.112105,0.354751,1502.302094,1537.223355,-3.71,4.0,2.55


In [10]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]
progress:: 418it [00:00, 4222.35it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-11-17,DET,IND,7.0,0.537601,0.463065,0.537268
1,2021-11-17,CHA,WAS,-1.0,0.502342,0.497706,0.502318
2,2021-11-17,MIA,NOP,-8.0,0.465992,0.533705,0.466143
3,2021-11-17,NYK,ORL,-12.0,0.469920,0.532124,0.468898
4,2021-11-17,MIL,LAL,-9.5,0.496098,0.502814,0.496642
5,2021-11-17,ATL,BOS,-4.5,0.473191,0.526089,0.473551
6,2021-11-17,BKN,CLE,-9.5,0.485807,0.515155,0.485326
7,2021-11-17,MIN,SAC,-2.0,0.505743,0.493017,0.506363
8,2021-11-17,OKC,HOU,-3.0,0.511800,0.489146,0.511327
9,2021-11-17,POR,CHI,-1.5,0.500961,0.498591,0.501185


In [9]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_sfs')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]
progress:: 418it [00:00, 4917.77it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-11-17,DET,IND,7.0,0.534500,0.485200,0.524650
1,2021-11-17,CHA,WAS,-1.0,0.510308,0.499850,0.505229
2,2021-11-17,MIA,NOP,-8.0,0.466756,0.544938,0.460909
3,2021-11-17,NYK,ORL,-12.0,0.474291,0.543660,0.465315
4,2021-11-17,MIL,LAL,-9.5,0.534843,0.479353,0.527745
5,2021-11-17,ATL,BOS,-4.5,0.473398,0.539577,0.466911
6,2021-11-17,BKN,CLE,-9.5,0.520235,0.493887,0.513174
7,2021-11-17,MIN,SAC,-2.0,0.499757,0.514346,0.492706
8,2021-11-17,OKC,HOU,-2.5,0.527863,0.487352,0.520256
9,2021-11-17,POR,CHI,-1.5,0.490494,0.517792,0.486351


In [11]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain1')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.12it/s]
progress:: 418it [00:00, 5160.66it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-11-17,DET,IND,7.0,0.532088,0.464613,0.533738
1,2021-11-17,CHA,WAS,-1.0,0.501723,0.492322,0.504701
2,2021-11-17,MIA,NOP,-8.0,0.455421,0.541767,0.456827
3,2021-11-17,NYK,ORL,-12.0,0.459101,0.541747,0.458677
4,2021-11-17,MIL,LAL,-9.5,0.522462,0.479064,0.521699
5,2021-11-17,ATL,BOS,-4.5,0.463560,0.531105,0.466228
6,2021-11-17,BKN,CLE,-9.5,0.504993,0.491907,0.506543
7,2021-11-17,MIN,SAC,-2.0,0.497421,0.496933,0.500244
8,2021-11-17,OKC,HOU,-3.0,0.513346,0.486057,0.513644
9,2021-11-17,POR,CHI,-1.5,0.487858,0.508700,0.489579
